In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shubhambaid/skin-burn-dataset")
print("Path to dataset files:", path)


100%|██████████| 16.6M/16.6M [00:01<00:00, 17.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/shubhambaid/skin-burn-dataset/versions/1


In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt


In [10]:
from glob import glob

# Example: Assuming dataset is structured as images and masks in separate folders
image_dir = os.path.join(path, "images")
mask_dir = os.path.join(path, "masks")

def load_data(img_paths, mask_paths, target_size=(128, 128)):
    X, Y = [], []
    for img_path, mask_path in zip(img_paths, mask_paths):
        img = load_img(img_path, target_size=target_size)
        img = img_to_array(img) / 255.0

        mask = load_img(mask_path, target_size=target_size, color_mode="grayscale")
        mask = img_to_array(mask) / 255.0

        X.append(img)
        Y.append(mask)

    return np.array(X), np.array(Y)

image_paths = sorted(glob(os.path.join(image_dir, "*.jpg")))
mask_paths = sorted(glob(os.path.join(mask_dir, "*.png")))

X, Y = load_data(image_paths, mask_paths)


In [14]:
import os

print("Dataset root path:", path)
print("Contents of dataset folder:")
print(os.listdir(path))


Dataset root path: /kaggle/input/skin-burn-dataset
Contents of dataset folder:
['img560.txt', 'img1194.jpg', 'img275.txt', 'img966.txt', 'img516.jpg', 'img943.txt', 'img254.jpg', 'img263.jpg', 'img699.txt', 'img659.jpg', 'img338.txt', 'img1387.txt', 'img575.jpg', 'img483.txt', 'img4.txt', 'img470.txt', 'img794.txt', 'img1013.jpg', 'img0.jpg', 'img988.txt', 'img66.txt', 'img1119.jpg', 'img403.jpg', 'img1138.jpg', 'img1280.txt', 'img1173.jpg', 'img786.txt', 'img522.txt', 'img1176.jpg', 'img696.txt', 'img1343.txt', 'img932.txt', 'img822.txt', 'img1120.txt', 'img224.jpg', 'img985.jpg', 'img113.txt', 'img514.jpg', 'img170.jpg', 'img212.jpg', 'img18.txt', 'img280.jpg', 'img344.txt', 'img341.txt', 'img1303.txt', 'img373.txt', 'img77.jpg', 'img340.jpg', 'img521.jpg', 'img154.jpg', 'img170.txt', 'img498.jpg', 'img226.txt', 'img42.jpg', 'img989.txt', 'img317.txt', 'img1144.txt', 'img921.txt', 'img1285.jpg', 'img1210.txt', 'img920.txt', 'img435.txt', 'img342.jpg', 'img691.jpg', 'img286.jpg', 'img

In [15]:
# Pick any mask .txt file to explore
with open("/kaggle/input/skin-burn-dataset/img560.txt") as f:
    content = f.read()

print("Sample content:\n", content)


Sample content:
 1 0.46653543307086615 0.5125628140703518 0.3425196850393701 0.5326633165829145



In [17]:
import numpy as np
from PIL import Image

def bbox_to_mask(bbox, img_width, img_height):
    # bbox = [class_id, x_center, y_center, width, height] normalized
    _, x_center, y_center, w, h = bbox

    # Convert normalized to pixel coordinates
    x_center *= img_width
    y_center *= img_height
    w *= img_width
    h *= img_height

    x_min = int(x_center - w/2)
    y_min = int(y_center - h/2)
    x_max = int(x_center + w/2)
    y_max = int(y_center + h/2)

    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    mask[y_min:y_max, x_min:x_max] = 1

    return mask

# Example usage
img_path = "/kaggle/input/skin-burn-dataset/img1194.jpg"
img = Image.open(img_path)
w, h = img.size

bbox = [1, 0.4665, 0.5125, 0.3425, 0.5326]  # example from txt content
mask = bbox_to_mask(bbox, w, h)

Image.fromarray(mask*255).show()  # show the mask image


In [18]:
import os
import numpy as np
from glob import glob
from PIL import Image
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# --- Paths ---
dataset_path = "/kaggle/input/skin-burn-dataset"
image_paths = sorted(glob(os.path.join(dataset_path, "*.jpg")))
txt_paths = sorted(glob(os.path.join(dataset_path, "*.txt")))

# --- Parameters ---
IMG_SIZE = 128

# --- Function to convert YOLO bbox txt annotation to mask ---
def bbox_to_mask(bbox, img_width, img_height):
    # bbox = [class_id, x_center, y_center, width, height] normalized
    _, x_center, y_center, w, h = bbox

    x_center *= img_width
    y_center *= img_height
    w *= img_width
    h *= img_height

    x_min = int(max(x_center - w / 2, 0))
    y_min = int(max(y_center - h / 2, 0))
    x_max = int(min(x_center + w / 2, img_width))
    y_max = int(min(y_center + h / 2, img_height))

    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    mask[y_min:y_max, x_min:x_max] = 1
    return mask

# --- Load one annotation file and convert all bboxes to one mask ---
def load_mask(txt_file, img_width, img_height):
    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    with open(txt_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            bbox = list(map(float, parts))
            single_mask = bbox_to_mask(bbox, img_width, img_height)
            mask = np.maximum(mask, single_mask)  # combine all bboxes
    return mask

# --- Load all data ---
def load_data(image_paths, txt_paths, img_size=IMG_SIZE):
    X, Y = [], []
    for img_path, txt_path in zip(image_paths, txt_paths):
        # Load and resize image
        img = load_img(img_path, target_size=(img_size, img_size))
        img_arr = img_to_array(img) / 255.0

        # Original image size for mask
        orig_img = Image.open(img_path)
        orig_w, orig_h = orig_img.size

        # Load and resize mask
        mask = load_mask(txt_path, orig_w, orig_h)
        mask_img = Image.fromarray(mask * 255).resize((img_size, img_size))
        mask_arr = np.array(mask_img) / 255.0
        mask_arr = np.expand_dims(mask_arr, axis=-1)  # add channel dim

        X.append(img_arr)
        Y.append(mask_arr)

    return np.array(X), np.array(Y)

# --- Build U-Net model ---
def unet(input_size=(IMG_SIZE, IMG_SIZE, 3)):
    inputs = Input(input_size)

    # Encoder
    c1 = Conv2D(16, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(16, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D()(c1)

    c2 = Conv2D(32, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(32, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D()(c2)

    c3 = Conv2D(64, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(64, 3, activation='relu', padding='same')(c3)
    p3 = MaxPooling2D()(c3)

    c4 = Conv2D(128, 3, activation='relu', padding='same')(p3)
    c4 = Conv2D(128, 3, activation='relu', padding='same')(c4)
    p4 = MaxPooling2D()(c4)

    # Bottleneck
    c5 = Conv2D(256, 3, activation='relu', padding='same')(p4)
    c5 = Conv2D(256, 3, activation='relu', padding='same')(c5)

    # Decoder
    u6 = Conv2DTranspose(128, 2, strides=2, padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, 3, activation='relu', padding='same')(u6)
    c6 = Conv2D(128, 3, activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(64, 2, strides=2, padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, 3, activation='relu', padding='same')(u7)
    c7 = Conv2D(64, 3, activation='relu', padding='same')(c7)

    u8 = Conv2DTranspose(32, 2, strides=2, padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, 3, activation='relu', padding='same')(u8)
    c8 = Conv2D(32, 3, activation='relu', padding='same')(c8)

    u9 = Conv2DTranspose(16, 2, strides=2, padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(16, 3, activation='relu', padding='same')(u9)
    c9 = Conv2D(16, 3, activation='relu', padding='same')(c9)

    outputs = Conv2D(1, 1, activation='sigmoid')(c9)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# --- Main ---

# Load data
X, Y = load_data(image_paths, txt_paths, IMG_SIZE)
print(f"Loaded {len(X)} samples.")

# Build and compile model
model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(X, Y, epochs=20, batch_size=8, validation_split=0.2)


Loaded 1225 samples.
Epoch 1/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.7587 - loss: 0.5674 - val_accuracy: 0.8415 - val_loss: 0.4290
Epoch 2/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.7740 - loss: 0.5008 - val_accuracy: 0.8415 - val_loss: 0.3690
Epoch 3/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.7790 - loss: 0.4330 - val_accuracy: 0.8407 - val_loss: 0.4031
Epoch 4/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.7835 - loss: 0.4318 - val_accuracy: 0.8414 - val_loss: 0.3677
Epoch 5/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.7825 - loss: 0.4285 - val_accuracy: 0.8367 - val_loss: 0.3776
Epoch 6/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.7811 - loss: 0.4318 - val_accuracy: 0.8220 - val_loss: 0.3681
Epoch 7/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 200s 1s/step - accuracy: 0.7808 - loss: 0.4286 - val_accuracy: 0.8064 - val_loss: 0.3849
Epoch 8/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.7919 - lo

In [19]:
# Print the final epoch accuracy
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Validation Accuracy: {val_acc:.4f}")


Final Training Accuracy: 0.7858
Final Validation Accuracy: 0.8050


In [20]:
Y_pred = model.predict(X)  # or X_test

# Threshold predicted masks at 0.5
Y_pred_bin = (Y_pred > 0.5).astype(np.uint8)

# Flatten and compare with ground truth masks
accuracy_manual = np.mean(Y_pred_bin.flatten() == Y.flatten())
print(f"Manual pixel-wise accuracy: {accuracy_manual:.4f}")


39/39 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step
Manual pixel-wise accuracy: 0.7885
